In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.callbacks import EarlyStopping

from utils.dataloader import load_data_npy, PreprocessedDataGenerator
from utils.model_utils import save_model_and_history

2025-05-27 20:03:12.863973: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 20:03:13.684632: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 20:03:14.173347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748376194.585229   53352 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748376194.736415   53352 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748376195.855342   53352 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
images_train, categories_train, images_val, categories_val, images_test, categories_test = load_data_npy()

# --- Generadores ---
def preprocess_fn(x):
    return vgg_preprocess(x)

# Generadores adaptados
train_gen = PreprocessedDataGenerator(images_train, categories_train, preprocess_fn=preprocess_fn, resize_to=(224, 224), batch_size=16)
val_gen = PreprocessedDataGenerator(images_val, categories_val, shuffle=False, preprocess_fn=preprocess_fn, resize_to=(224, 224), batch_size=16)
test_gen = PreprocessedDataGenerator(images_test, categories_test, shuffle=False, preprocess_fn=preprocess_fn, resize_to=(224, 224), batch_size=16)

print(f"Train: {images_train.shape}, Validation: {images_val.shape}, Test: {images_test.shape}")

Train: (10220, 150, 150, 3), Validation: (2555, 150, 150, 3), Test: (4259, 150, 150, 3)


In [ ]:
# Opción 2: VGG16 (más pesado, buena extracción de características)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar capas convolucionales del modelo base
base_model.trainable = False  # para feature extraction

# --- Construir modelo con cabeza personalizada ---
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# --- Entrenamiento ---
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=30,
                    callbacks=[early_stop])

# --- Evaluación ---
test_loss, test_acc = model.evaluate(test_gen)
print(f"\nTest Accuracy: {test_acc:.4f}   |  Test Loss: {test_loss:.4f}")

# --- Guardado ---
save_model_and_history(model, history, model_path='transfer_model_vgg16')

2025-05-27 20:05:20.676298: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 3991s 6s/step - accuracy: 0.1893 - loss: 1.9563 - val_accuracy: 0.2583 - val_loss: 1.7535
Epoch 2/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 4595s 7s/step - accuracy: 0.2271 - loss: 1.7463 - val_accuracy: 0.3088 - val_loss: 1.6730
Epoch 3/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 4320s 7s/step - accuracy: 0.2629 - loss: 1.6853 - val_accuracy: 0.3507 - val_loss: 1.6178
Epoch 4/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 4337s 7s/step - accuracy: 0.2880 - loss: 1.6342 - val_accuracy: 0.3115 - val_loss: 1.5762
Epoch 5/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 4195s 7s/step - accuracy: 0.3023 - loss: 1.6025 - val_accuracy: 0.3523 - val_loss: 1.5388
Epoch 6/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 3951s 6s/step - accuracy: 0.3168 - loss: 1.5806 - val_accuracy: 0.3609 - val_loss: 1.5135
Epoch 7/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 3144s 5s/step - accuracy: 0.3322 - loss: 1.5425 - val_accuracy: 0.3644 - val_loss: 1.4755
Epoch 8/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 2903s 4s/step - accuracy: 0.3374 - loss: 1.5200 - 